# Extended Events

***You will need to have followed the steps in the 00-CreateContainers notebook to use this notebook***

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

dbatools is **awesome** with Extended Events

First we will set up the variables and connections to the containers

In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}
# Run this first to make sure output width does not mess with output - Update output buffer size to prevent clipping in Visual Studio output window.
if( $Host -and $Host.UI -and $Host.UI.RawUI ) {
    $rawUI = $Host.UI.RawUI
    $oldSize = $rawUI.BufferSize
    $typeName = $oldSize.GetType( ).FullName
    $newSize = New-Object $typeName (500, $oldSize.Height)
    $rawUI.BufferSize = $newSize
  }

## Listing Extended Events

Which Extended Event sessions do we have on our instances ?

This will give you 

- The name of the session
- If it is running
- When it started
- If it will start on server startup (Autostart = True)
- The targets
- The target filename
- The events

In [ ]:
Get-DbaXESession -SqlInstance $SQL1

## Starting and stopping Extended Events Sessions

It is no surprise that you can start and stop extended events sessions easily with dbatools. We like to make command names easy to remember!

`Start-DbaXeSession`  
`Stop-DbaXeSession`

The following session is running - Yep we can check a session with `Get-DbaXeSession` - Look at the status to see if it is running

In [ ]:
Get-DbaXESession -SqlInstance $SQL1 -Session 'system_health'

We can stop it


In [ ]:
Stop-DbaXESession  -SqlInstance $SQL1 -Session 'system_health'

and start it again


In [ ]:
Start-DbaXESession  -SqlInstance $SQL1 -Session 'system_health'

Lets add a new session. Notoebooks dont like the XE dlls so we have to open a PowerShell window seperately.

In [ ]:
 ## When PowerShell opens - right click to paste the command
# 
## the password is dbatools.IO
" Import-DbaXESessionTemplate -SqlInstance 'localhost,15592' -SqlCredential sqladmin -Template 'Blocked Process Report'" | clip

Start-Process 'C:\Windows\system32\WindowsPowerShell\v1.0\powershell.exe'

In [ ]:
Start-DbaXESession  -SqlInstance $SQL1 -Session 'Blocked Process Report'

We can watch the live data - Lets create a different process to watch the Blocked Process Session.

This code will open a Windows PowerShell session, right click to paste the code and it will prompt for a password which is `dbatools.IO` You will need to have installed dbatools in Windows PowerShell prior to this.
Leave that running and move to the next step

In [ ]:
Set-DbaSpConfigure -SqlInstance $sql1 -Name 'show advanced options' -Value 1
Set-DbaSpConfigure -SqlInstance $sql1 -Name 'BlockedProcessThreshold' -Value 5
Set-DbaSpConfigure -SqlInstance $sql1 -Name 'show advanced options' -Value 0


In [ ]:
## When PowerShell opens - right click to paste the command
#  
@"
## the password is dbatools.IO
Get-DbaXESession -SqlInstance 'localhost,15592' -SqlCredential sqladmin -Session 'Blocked Process Report' | Watch-DbaXESession  
"@ | clip

# 
Start-Process 'C:\Windows\system32\WindowsPowerShell\v1.0\powershell.exe'

This code will create some blocked processes but only in a dotnet notebook, once you have run it you can look at the Windows PowerShell window and see the details, close the window when you are done!

In [ ]:
$query1 = "
BEGIN TRANSACTION
  SELECT * FROM [dbo].[authors] WITH (TABLOCKX, HOLDLOCK)
    WAITFOR DELAY '00:00:30' -- 30 seconds
ROLLBACK TRANSACTION
"
$query2 = "SELECT TOP 1 * FROM [dbo].[authors]" 
$Queries = $query1, $query2, $query2 , $query2, $query2, $query2, $query2, $query2, $query2, $query2, $query2, $query2, $query2, $query2
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$Queries |ForEach-Object   {
$Credential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbaQuery -SqlInstance 'localhost,15592' -SqlCredential $Credential -Database pubs -Query $psitem 
}

Thats really cool, you can see the data flying by on the screen  
  
![dbatools](.\images\blockedprocessXE.png )  

But what about if you would like to read the data

There is no need to learn the XML shredding :-)

We can read the file.

First we need to know where the file is

In [ ]:
Get-DbaXESessionTarget -SqlInstance $SQL1 -Session 'Blocked Process Report'

If we weren't using containers you could use `Get-DbaXeSessionTargetFile` and if you were on Windows and had access to the admin UNC share, you could do

`Get-DbaXESession -SqlInstance $SQL1 -Session 'Blocked Process Report' | Read-DbaXEFile`

But we arent so lucky. Lets copy the files to the directory we have mounted for our containers (I copy all the files because after 5 minutes of trying to only copy the xel files with bash I gave up)

In [ ]:
docker exec -i beardsql01 cp -R /var/opt/mssql/log/ /var/opt/mssql/backups/

With the files moved to a place we can access them, we can read the files with one line of code. It won't work in the notebook but the code below will open a Windows PowerShell session and the you can right click to paste the command and read the data. `CTRL + C ` will stop it and you can just close the PowerShell session when it is done

In [ ]:
## When PowerShell opens - right click to paste the command
#     
"Get-ChildItem $FolderPath\log\Block* | Read-DbaXEFile" | clip

Start-Process 'C:\Windows\system32\WindowsPowerShell\v1.0\powershell.exe'

But we are using PowerShell which means we have all of the tools available to us :-)

How about if we want the XE event data in a CSV file ?

Again we need to open a PowerShell session and right click and when it is done, close it and come back here

In [ ]:
## When PowerShell opens - right click to paste the command
#     
"Get-ChildItem $FolderPath\log\Block* | Read-DbaXEFile | Export-Csv -Path $FolderPath\BlockedProcess.csv -NoClobber" | clip

Start-Process 'C:\Windows\system32\WindowsPowerShell\v1.0\powershell.exe'

Now we can read the CSV file


In [ ]:
$BlockedProcesses = Import-Csv $FolderPath\BlockedProcess.csv 
$BlockedProcesses | Select -First 1

We could even put them in a database using 

`$BlockedProcesses | Write-DbaDataTable -SqlInstance $SQL1 -Database tempdb -Table BlockedProcess -AutoCreateTable`

or an Excel File with ImportExcel, Email, Word Document, back to XML, into Azure etc etc

# New Extended Events

OK thats all very well and good, I can see, start, stop, watch, read the files with dbatools but I want to create a new one. dbatools has you covered.

We have templates built in from Microsoft and Community members like Erin Stellato, Jes Borland, Ola Hallengren and more (Thank you all very much for this )

In [ ]:
Get-DbaXESessionTemplate  | Select Category, Compatibility , Name, Source

You can see exactly what it will do

In [ ]:
Get-DbaXESessionTemplate -Template 'Blocked Process Report'

So you can pick a ready made session and add it to your instance easily. Again, notebooks dont like the XE.dlls for some reason so we have to open a Windows PowerShell Session

In [ ]:
 ## When PowerShell opens - right click to paste the command
# 
## the password is dbatools.IO
" Import-DbaXESessionTemplate -SqlInstance 'localhost,15592' -SqlCredential sqladmin -Template 'Index Page Splits'" | clip

Start-Process 'C:\Windows\system32\WindowsPowerShell\v1.0\powershell.exe'

If we look at the instance for that session

In [ ]:
Get-DbaXESession -SqlInstance $SQL1 -Session 'Index Page Splits'

In [ ]:
Get-DbaTrace -SqlInstance $sql1 # | Out-GridView -PassThru | ConvertTo-DbaXESession -Name 'Test' | Start-DbaXESession

There is so much more that dbatools can do with Extended Events take a look at Chrissy and Gianluca's presentation from SQL Bits

Video - https://sqlbits.com/Sessions/Event17/Simplifying_XEvents_Management_with_dbatools  
Code - https://github.com/sqlcollaborative/community-presentations/tree/master/chrissy-lemaire-gianluca-sartori/bits-xevents

# Clean Up

The 99-CleanUpContainers notebook will remove the containers, files and directory - it will leave the image so you do not have to download it again!